In [56]:
!pip install tabula-py
import tabula
import pandas as pd
import numpy as np

In [57]:
table = tabula.read_pdf("G1.pdf", pages=1)

In [58]:
for i in range(len(table)):
    table[i].columns = table[i].columns.str.replace(r"\r", " ")

In [59]:
table[0] = table[0].drop(columns=["Unnamed: 0"])

In [60]:
index = table[0].index
condition1 = table[0]['CHICOPEE BIG Y'] == "WEEKDAY"
condition2 = table[0]['CHICOPEE BIG Y'] == 'SATURDAY'
condition3 = table[0]['CHICOPEE BIG Y'] == 'SUNDAY'
day_indices = index[condition1][0], index[condition2][0], index[condition3][0]

table_weekday = table[0].iloc[day_indices[0] + 1: day_indices[1] - 1]
table_saturday = table[0].iloc[day_indices[1] + 1: day_indices[2] - 1]
table_sunday = table[0].iloc[day_indices[2] + 1: len(table[0])]
table_weekday.reset_index(drop=True, inplace=True)
table_saturday.reset_index(drop=True, inplace=True)
table_sunday.reset_index(drop=True, inplace=True)

In [61]:
def preprocess_table(table):
    for i in range(len(table.columns)):
        col = table.columns[i]
        table[col] = table[col].str.replace('[\|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|-]','')

In [62]:
preprocess_table(table_weekday)
preprocess_table(table_saturday)
preprocess_table(table_sunday)

C:\Users\quynh\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [63]:
# condition on mornings for weekdays. 
condition_weekday_morning = table_weekday['CHICOPEE BIG Y'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_weekday_morning = table_weekday.iloc[0: condition_weekday_morning]
table_weekday_morning.reset_index(drop=True, inplace=True)

In [64]:
# condition on evenings for weekdays
condition_weekday_evening = table_weekday['CHICOPEE BIG Y.1'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_weekday_evening = table_weekday.iloc[condition_weekday_evening - 1:]
table_weekday_evening.reset_index(drop=True, inplace=True)

In [65]:
# condition on mornings for saturdays. 
condition_saturday_morning = table_saturday['CHICOPEE BIG Y'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_saturday_morning = table_saturday.iloc[0: condition_saturday_morning]
table_saturday_morning.reset_index(drop=True, inplace=True)

In [66]:
# condition on evenings for saturdays. 
condition_saturday_evening = table_saturday['CHICOPEE BIG Y.1'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_saturday_evening = table_saturday.iloc[condition_saturday_evening - 1:]
table_saturday_evening.reset_index(drop=True, inplace=True)

In [67]:
# condition on mornings for sunday. 
condition_sunday_morning = table_sunday['CHICOPEE BIG Y'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_sunday_morning = table_sunday.iloc[0: condition_sunday_morning]
table_sunday_morning.reset_index(drop=True, inplace=True)

In [68]:
# condition on evenings for sunday. 
condition_sunday_evening = table_sunday['CHICOPEE BIG Y.1'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_sunday_evening = table_sunday.iloc[condition_sunday_evening - 1:]
table_sunday_evening.reset_index(drop=True, inplace=True)

In [70]:
def preprocess_times_morning(table):
    for i in range(int(len(table.columns)/2), len(table.columns)):
        if i != 0:
            col = table.columns[i]
            index = table[col].str.contains('[1][2]:[0-5][0-9]').idxmax()
            table.loc[index:,col] = ''
def preprocess_times_evening(table):
    for i in range(0, int(len(table.columns)/2)):
        col = table.columns[i]
        index = table[col].str.contains('[1][2]:[0-5][0-9]').idxmax()
        if index != -1:
            table.loc[0:index-1,col] = ''

In [71]:
table_weekday_evening = table_weekday_evening.copy()
preprocess_times_evening(table_weekday_evening)
table_saturday_evening = table_saturday_evening.copy()
preprocess_times_evening(table_saturday_evening)
table_sunday_evening = table_sunday_evening.copy()
preprocess_times_evening(table_sunday_evening)
table_weekday_morning = table_weekday_morning.copy()
preprocess_times_morning(table_weekday_morning)
table_saturday_morning = table_saturday_morning.copy()
preprocess_times_morning(table_saturday_morning)
table_sunday_morning = table_sunday_morning.copy()
preprocess_times_morning(table_sunday_morning)

In [72]:
table_weekday_morning

,CHICOPEE BIG Y,CHICOPEE CENTER,UNION STATION BAY 2,"THE ""X""",RIVERBEND MEDICAL CENTER,ALLEN & COOLEY/ 5 TOWN PLAZA,CANON CIRCLE,ALLEN & COOLEY/ 5 TOWN PLAZA.1,"THE ""X"".1",UNION STATION BAY 6,CHICOPEE CENTER.1,CHICOPEE BIG Y.1
0,,,,,,,,,,,6:00,6:18
1,,,,,,,,5:35,5:45,6:10,6:25,6:43
2,,,,,,,,,,6:25,6:45,7:03
3,,,6:00,6:10,,,6:30,6:35,6:45,7:15,7:35,7:58
4,,,6:15,6:25,,,,,6:30,6:55,7:15,7:38
5,,,6:25,6:35,6:50,6:55,7:00,,7:10,7:40,8:00,8:23
6,,6:25,6:45,6:55,,,7:15,7:20,7:30,8:00,8:20,8:43
7,6:30,6:40,7:00,7:15,,,7:35,7:40,7:50,8:20,8:35,
8,6:45,7:00,7:20,7:35,7:45,7:55,8:00,,8:10,8:40,8:55,9:18
9,7:05,7:20,7:40,7:55,,,8:15,8:20,8:30,9:00,9:15,


In [73]:
table_weekday_evening

,CHICOPEE BIG Y,CHICOPEE CENTER,UNION STATION BAY 2,"THE ""X""",RIVERBEND MEDICAL CENTER,ALLEN & COOLEY/ 5 TOWN PLAZA,CANON CIRCLE,ALLEN & COOLEY/ 5 TOWN PLAZA.1,"THE ""X"".1",UNION STATION BAY 6,CHICOPEE CENTER.1,CHICOPEE BIG Y.1
0,,,,,,,10:35,10:40,10:50,11:20,11:35,
1,,,,,,,11:00,,11:10,11:40,11:55,12:18
2,,,,,,,11:15,11:20,11:30,12:00,12:15,
3,,,,,,,11:40,,11:50,12:20,12:35,12:58
4,,,,,,,11:55,12:00,12:10,12:40,12:55,
5,,,,,12:05,12:15,12:20,,12:30,1:00,1:15,1:38
6,,,12:00,12:15,,,12:35,12:40,12:50,1:20,1:35,
7,,12:00,12:20,12:35,12:45,12:55,1:00,,1:10,1:40,1:55,2:18
8,,12:20,12:40,12:55,,,1:15,1:20,1:30,2:00,2:15,
9,12:25,12:40,1:00,1:15,1:25,1:35,1:40,,1:50,2:20,2:35,2:58


In [26]:
table_weekday_morning.to_csv(r'./PVTA_Route_Data/weekday_morning.csv', index=False, header=True)
table_weekday_evening.to_csv(r'./PVTA_Route_Data/weekday_evening.csv', index=False, header=True)
table_saturday_morning.to_csv(r'./PVTA_Route_Data/saturday_morning.csv', index=False, header=True)
table_saturday_evening.to_csv(r'./PVTA_Route_Data/saturday_evening.csv', index=False, header=True)
table_sunday_morning.to_csv(r'./PVTA_Route_Data/sunday_morning.csv', index=False, header=True)
table_sunday_evening.to_csv(r'./PVTA_Route_Data/sunday_evening.csv', index=False, header=True)

In [ ]:
'''
table_weekday_morning = table_weekday_morning.replace(r'^\s*$', np.nan, regex=True)
table_weekday_evening = table_weekday_evening.replace(r'^\s*$', np.nan, regex=True)
table_saturday_morning = table_saturday_morning.replace(r'^\s*$', np.nan, regex=True)
table_saturday_evening = table_saturday_evening.replace(r'^\s*$', np.nan, regex=True)
table_sunday_morning = table_sunday_morning.replace(r'^\s*$', np.nan, regex=True)
table_sunday_evening = table_sunday_evening.replace(r'^\s*$', np.nan, regex=True)

# Drop apply
table_weekday_morning = table_weekday_morning.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_weekday_evening = table_weekday_evening.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_saturday_morning = table_saturday_morning.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_saturday_evening = table_saturday_evening.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_sunday_morning = table_sunday_morning.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_sunday_evening = table_sunday_evening.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
'''